In [1]:
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.cluster import AffinityPropagation, DBSCAN, KMeans, MiniBatchKMeans
#import cytoolz
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.sparse import vstack
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import random
from sklearn.decomposition import PCA

In [2]:
#read the doc2vec file with the 
labeled_file=pd.read_csv('/home/ubuntu/git/word2vec_exp/labeled_data_doc2vec.csv')

In [3]:
#verify it worked
labeled_file.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,id
0,0.009416,0.068022,-0.093844,-0.025640,0.022005,-0.059202,0.019508,-0.000502,-0.073609,-0.043668,...,0.041896,0.005071,-0.008157,0.005333,-0.015944,0.038117,0.099916,-0.069594,-0.111508,"""5814_8"""
1,-0.031307,0.022234,-0.038260,-0.023016,0.037632,-0.045624,-0.018393,-0.063229,-0.073714,-0.072424,...,0.034739,-0.025787,-0.016142,0.017052,-0.033065,0.029010,0.042111,-0.014770,-0.075852,"""2381_9"""
2,0.032531,0.000934,-0.124349,-0.085180,-0.052381,0.012115,0.004403,0.023709,0.057304,-0.027850,...,0.031160,-0.007705,0.054912,-0.024911,-0.054330,-0.023962,0.019848,-0.117262,-0.110720,"""7759_3"""
3,-0.028597,0.006379,-0.051248,-0.065585,0.020525,-0.052478,-0.040403,-0.078019,-0.028415,-0.071265,...,0.050628,-0.030066,-0.005953,0.019702,-0.069980,0.002639,0.026106,-0.050067,-0.087348,"""3630_4"""
4,0.018592,0.043664,-0.132684,-0.057355,-0.021617,-0.036612,0.017030,0.008483,-0.017411,-0.063672,...,0.033910,0.001192,0.035522,0.001874,-0.043808,0.006309,0.072873,-0.102422,-0.156285,"""9495_8"""


In [4]:
#first step is to reduce the number of dimensions of the document vectors
#labeled_file['doc_vec'].values
labeled_2=labeled_file.iloc[:,0:300]
labeled_2.head()
#create a clustering algorithm
#labeled_file['doc_vec'].to


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.009416,0.068022,-0.093844,-0.025640,0.022005,-0.059202,0.019508,-0.000502,-0.073609,-0.043668,...,-0.008266,0.041896,0.005071,-0.008157,0.005333,-0.015944,0.038117,0.099916,-0.069594,-0.111508
1,-0.031307,0.022234,-0.038260,-0.023016,0.037632,-0.045624,-0.018393,-0.063229,-0.073714,-0.072424,...,-0.016956,0.034739,-0.025787,-0.016142,0.017052,-0.033065,0.029010,0.042111,-0.014770,-0.075852
2,0.032531,0.000934,-0.124349,-0.085180,-0.052381,0.012115,0.004403,0.023709,0.057304,-0.027850,...,-0.112527,0.031160,-0.007705,0.054912,-0.024911,-0.054330,-0.023962,0.019848,-0.117262,-0.110720
3,-0.028597,0.006379,-0.051248,-0.065585,0.020525,-0.052478,-0.040403,-0.078019,-0.028415,-0.071265,...,-0.086711,0.050628,-0.030066,-0.005953,0.019702,-0.069980,0.002639,0.026106,-0.050067,-0.087348
4,0.018592,0.043664,-0.132684,-0.057355,-0.021617,-0.036612,0.017030,0.008483,-0.017411,-0.063672,...,-0.076063,0.033910,0.001192,0.035522,0.001874,-0.043808,0.006309,0.072873,-0.102422,-0.156285


In [7]:
def pca_data_frame(dataframe,no_components):
    
    #clustering with pca
    dataframe=dataframe.iloc[:,0:300]
    #run PCA
    pca = PCA(n_components=no_components)  
    x_pca=pca.fit_transform(dataframe)
    rat=pca.explained_variance_ratio_
    print(np.sum(pca.explained_variance_ratio_))
    return x_pca,rat
    

In [14]:
#create a dataframe composed of 30 principal components. These explain 99 percent of the dataset variation.
dat=pca_data_frame(labeled_file,30)


0.994301938235


In [16]:
#separate the components
dat_cluster=dat[0]
type(dat_cluster)
np.shape(dat_cluster)

(24904, 30)

In [18]:
#we are now ready to run the clustering algorithm
sub_engine = MiniBatchKMeans(n_clusters = 5, init = 'k-means++',
                             batch_size = 1000,
                             max_no_improvement = 5, verbose = 2)

In [19]:
clusters_mini=sub_engine.fit(dat_cluster)

Init 1/3 with method: k-means++
Inertia for init 1/3: 412.988447
Init 2/3 with method: k-means++
Inertia for init 2/3: 412.578845
Init 3/3 with method: k-means++
Inertia for init 3/3: 403.399370
Minibatch iteration 1/2500: mean batch inertia: 0.107180, ewa inertia: 0.107180 
Minibatch iteration 2/2500: mean batch inertia: 0.103900, ewa inertia: 0.106916 
Minibatch iteration 3/2500: mean batch inertia: 0.091746, ewa inertia: 0.105698 
Minibatch iteration 4/2500: mean batch inertia: 0.100030, ewa inertia: 0.105243 
Minibatch iteration 5/2500: mean batch inertia: 0.183073, ewa inertia: 0.111493 
Minibatch iteration 6/2500: mean batch inertia: 0.112642, ewa inertia: 0.111585 
Minibatch iteration 7/2500: mean batch inertia: 0.112137, ewa inertia: 0.111630 
Minibatch iteration 8/2500: mean batch inertia: 0.098958, ewa inertia: 0.110612 
Minibatch iteration 9/2500: mean batch inertia: 0.086597, ewa inertia: 0.108684 
Converged (lack of improvement in inertia) at iteration 9/2500
Computing lab

In [22]:
labels_clusters=clusters_mini.labels_
len(labels)

24904

In [21]:
labels_clusters

array([0, 0, 2, ..., 1, 2, 4], dtype=int32)

In [27]:
labeled_file['cluster']=labels_clusters

In [28]:
labeled_file.head()

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,id,cluster
0,0.009416,0.068022,-0.093844,-0.025640,0.022005,-0.059202,0.019508,-0.000502,-0.073609,-0.043668,...,0.005071,-0.008157,0.005333,-0.015944,0.038117,0.099916,-0.069594,-0.111508,"""5814_8""",0
1,-0.031307,0.022234,-0.038260,-0.023016,0.037632,-0.045624,-0.018393,-0.063229,-0.073714,-0.072424,...,-0.025787,-0.016142,0.017052,-0.033065,0.029010,0.042111,-0.014770,-0.075852,"""2381_9""",0
2,0.032531,0.000934,-0.124349,-0.085180,-0.052381,0.012115,0.004403,0.023709,0.057304,-0.027850,...,-0.007705,0.054912,-0.024911,-0.054330,-0.023962,0.019848,-0.117262,-0.110720,"""7759_3""",2
3,-0.028597,0.006379,-0.051248,-0.065585,0.020525,-0.052478,-0.040403,-0.078019,-0.028415,-0.071265,...,-0.030066,-0.005953,0.019702,-0.069980,0.002639,0.026106,-0.050067,-0.087348,"""3630_4""",2
4,0.018592,0.043664,-0.132684,-0.057355,-0.021617,-0.036612,0.017030,0.008483,-0.017411,-0.063672,...,0.001192,0.035522,0.001874,-0.043808,0.006309,0.072873,-0.102422,-0.156285,"""9495_8""",0


In [ ]:
#plot the tsnet plot based on the clusters as color and using the pca aplied to the first 60 components
labeld_file

In [27]:
dat=pca_data_frame(labeled_file,60)
#len(dat)
#print (dat[1])
type(dat[1])
x_pca=pca.fit_transform(dataframe)

dataframe=dataframe.iloc[:,0:300]
    #run PCA
    pca = PCA(n_components=no_components)  
    x_pca=pca.fit_transform(dataframe)
    rat=pca.explained_variance_ratio_

array([[  2.07745578e-01,  -5.37473633e-01,   5.51833818e-02, ...,
          1.50757211e-02,   6.87716397e-03,   3.57660399e-03],
       [  2.43847198e-02,  -2.00262520e-01,  -2.96405530e-01, ...,
         -4.94398116e-04,  -6.35957156e-04,  -3.88555771e-04],
       [  6.25628898e-01,   1.43754818e-01,   4.42588086e-01, ...,
          3.36547127e-05,  -4.15956840e-05,   3.31751764e-03],
       ..., 
       [ -3.85317580e-01,   6.95568306e-02,   9.59130737e-02, ...,
         -7.76607578e-04,   1.25225739e-03,  -4.13761271e-04],
       [  3.89742808e-01,  -4.46371713e-02,  -1.70681253e-01, ...,
         -2.50915930e-04,   9.24691508e-05,  -7.96840186e-04],
       [ -2.40377906e-01,  -1.47495283e-02,  -1.70452614e-02, ...,
          3.04208181e-03,   8.90423670e-04,   2.12816981e-03]])

In [ ]:
#implement kmeans clusters


In [ ]:
#implement tsnet